<a href="https://colab.research.google.com/github/kaushalkuma-r/Deep-Learning-Algorithms/blob/main/INTER_IIT_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import re
import requests
import urllib
from bs4 import BeautifulSoup
import pandas as pd

In [66]:
def make_url(base_url , comp):
    
    url = base_url
    
    # add each component to the base url
    for r in comp:
        url = '{}/{}'.format(url, r)
        
    return url

In [67]:
year_list=['2019','2020','2021','2018','2017','2016','2015','2014','2013','2012','2011','2010','2009','2008','2007','2006','2005','2004']
hdr= {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}

In [68]:
list_url=[]
def scraping_files(year1):
  
# define the urls needed to make the request, let's start with all the daily filings
  base_url = r"https://www.sec.gov/Archives/edgar/full-index"


  # The daily-index filings, requir/e a year and content type (html, json, or xml).
  year_url = make_url(base_url, [year1, 'index.json'])

# Display the new Year URL
  # print('-'*100)
  # print('Building the URL for Year: {}'.format(year))
  print("URL Link: " + year_url)

# request the content for 2019, remember that a JSON strucutre will be sent back so we need to decode it.
  content = requests.get(year_url,headers=hdr)
  print(content)
  decoded_content = content.json()

# the structure is almost identical to other json requests we've made. Go to the item list.
# AGAIN ONLY GRABBING A SUBSET OF THE FULL DATASET 
  for item in decoded_content['directory']['item'][0:1]:
    
    # get the name of the folder
    print('-'*100)
    print('Pulling url for Quarter: {}'.format(item['name']))
    
    # The daily-index filings, require a year, a quarter and a content type (html, json, or xml).
    qtr_url = make_url(base_url, [year1, item['name'], 'index.json'])
    
    # print out the url.
    print("URL Link: " + qtr_url)
    
    # Request, the new url and again it will be a JSON structure.
    file_content = requests.get(qtr_url,headers=hdr)
    decoded_content = file_content.json()
    
    # print('-'*100)
    # print('Pulling files')

    # for each file in the directory items list, print the file type and file href.
    # AGAIN DOING A SUBSET
    for file in decoded_content['directory']['item'][0:20]:
        
        file_url = make_url(base_url, [year1, item['name'], file['name']])
        list_url.append(file_url)
        # print("File URL Link: " + file_url)

In [69]:
for i in range(len(year_list)):
  scraping_files(year_list[i])

URL Link: https://www.sec.gov/Archives/edgar/full-index/2019/index.json
<Response [403]>


JSONDecodeError: ignored

In [ ]:
list_url

In [ ]:
df=pd.read_excel("Company.xlsx")
companies_name_list=df['Company'].tolist()
company_first=[]
for i in companies_name_list:
  l1=i.split(' ')
  company_first.append(l1[0])
form_type=['10-K','8-K','10-Q']

In [ ]:
len(companies_name_list)

In [ ]:
print(company_first)

#### Master list to store the url of required companies and the element is of the form ( CIK, Company Name, Form Type, Year and Quarter, url)

> Indented block



In [ ]:
master_data = []

def parser_idx(url):
  file_url = r'%s' % url

  # request that new content, this will not be a JSON STRUCTURE!
  content = requests.get(file_url,headers=hdr).content

# we can always write the content to a file, so we don't need to request it again.
  with open('master_20190102.txt', 'wb') as f:
       f.write(content)
  # let's open it and we will now have a byte stream to play with.
  with open('master_20190102.txt','rb') as f:
     byte_data = f.read()

# Now that we loaded the data, we have a byte stream that needs to be decoded and then split by double spaces.
  data = byte_data.decode("utf-8").split('  ')

# We need to remove the headers, so look for the end of the header and grab it's index
  for index, item in enumerate(data):
    if "ftp://ftp.sec.gov/edgar/" in item:
      start_ind = index

# define a new dataset with out the header info.
  data_format = data[start_ind + 1:]

  

# now we need to break the data into sections, this way we can move to the final step of getting each row value.
  for index, item in enumerate(data_format):
    
    # if it's the first index, it won't be even so treat it differently
    if index == 0:
        clean_item_data = item.replace('\n','|').split('|')
        clean_item_data = clean_item_data[8:]
    else:
        clean_item_data = item.replace('\n','|').split('|')
        
    for index, row in enumerate(clean_item_data):
        
        # when you find the text file.
        if '.txt' in row:

            # grab the values that belong to that row. It's 4 values before and one after.
            mini_list = clean_item_data[(index - 4): index + 1]
            # l1=mini_list[1].split(',')
            
                        
            # if len(mini_list) != 0 and (mini_list[1] in companies_name_list) :
            if len(mini_list) != 0 and (mini_list[2] in form_type) :
                mini_list[4] = "https://www.sec.gov/Archives/" + mini_list[4]
                master_data.append(mini_list)
                
  
# grab the first three items
  master_data[:3]
  

In [ ]:
for i in list_url:
  i_length=len(i)
  if i[i_length-10:i_length]=="master.idx":
    try:
      parser_idx(i)
    except:
      pass


### Master list to store the data. Each element is in the form (CIK, Company Name, Form Type, Date, URL for file)

In [ ]:
master_dir=[]
for i in master_data:
  if i[1] in company_first or i[1] in companies_name_list:
    master_dir.append(i)
  
print(master_dir)

In [ ]:
len(master_dir)
final_company=[]
for i in master_dir:
  if i[1] not in final_company:
    final_company.append(i[1])

# New section

In [ ]:
for index, document in enumerate(master_dir):
    
    # create a dictionary for each document in the master list
    document_dict = {}
    document_dict['cik_number'] = document[0]
    document_dict['company_name'] = document[1]
    document_dict['form_id'] = document[2]
    document_dict['date'] = document[3]
    document_dict['file_url'] = document[4]
    if document[1] in companies_name_list or document[1] in company_first:
      master_dir[index] = document_dict
    


In [ ]:
for i in master_data:
  if i['form_id'] not in form_type:
    master_data.remove(i)

In [ ]:
len(master_dir)

In [ ]:
master_dir[0:10]

#### Function to parse 10-K


In [ ]:
def extraction_10K():
  # create the list to hold the statement urls
  statements_url = []

  for report_dict in master_reports:
    
    # define the statements we want to look for.
    item1 = r"Consolidated Balance Sheets"
    item2 = r"Consolidated Statements of Operations and Comprehensive Income (Loss)"
    item3 = r"Consolidated Statements of Cash Flows"
    item4 = r"Consolidated Statements of Stockholder's (Deficit) Equity"
    
    # store them in a list.
    report_list = [item1, item2, item3, item4]
    
    # if the short name can be found in the report list.
    if report_dict['name_short'] in report_list:
        
        # print some info and store it in the statements url.
        print('-'*100)
        print(report_dict['name_short'])
        print(report_dict['url'])
        
        statements_url.append(report_dict['url'])